In [2]:
import os
import time
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
import math
from sklearn.model_selection import train_test_split
from sklearn import metrics

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, CuDNNGRU, Conv1D
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers

Using TensorFlow backend.


In [10]:
train = pd.read_csv("../input/quora-insincere-questions-classification/train.csv")
test_df = pd.read_csv("../input/quora-insincere-questions-classification/test.csv")
# print("Train shape : ",train_df.shape)
print("Test shape : ",test_df.shape)

Test shape :  (375806, 2)


In [1]:
train.head()

NameError: name 'train' is not defined

In [5]:
train_df = pd.read_csv("../input/quora1/X_train (1).csv")
val_df = pd.read_csv("../input/quora1/X_val .csv")
print("Train shape : ",train_df.shape)
print("Test shape : ",val_df.shape)

Train shape :  (162532, 9)
Test shape :  (261225, 9)


In [9]:
train_df.head()
# train_df['target'].value_counts()


,Unnamed: 0,question_text,num_words,num_unique_words,num_chars,num_words_upper,num_words_title,mean_word_len,target
0,330096,"Was it really an ""accident"" that Buddy, the Cl...",41,37,236,0,5,4.780488,1
1,148377,Why is Quora full of self-obsessed and narciss...,9,9,59,0,2,5.666667,1
2,605482,How does Jimmy Wales feel about Wikipedia stat...,16,16,105,0,4,5.625000,1
3,278650,Is it true that Quora represents the Jews’ com...,9,9,53,0,3,5.000000,1
4,669320,How common is it when a man publicly disagrees...,25,22,146,0,1,4.880000,1


In [ ]:
## split to train and val
# train_df, val_df = train_test_split(train_df, test_size=0.1, random_state=2018)


In [8]:
## some config values 
embed_size = 300 # how big is each word vector
max_features = 50000 # how many unique words to use (i.e num rows in embedding vector)
maxlen = 100 # max number of words in a question to use


In [11]:
## fill up the missing values
train_X = train_df["question_text"].fillna("_na_").values
val_X = val_df["question_text"].fillna("_na_").values
test_X = test_df["question_text"].fillna("_na_").values


In [12]:
## Tokenize the sentences
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(train_X))
train_X = tokenizer.texts_to_sequences(train_X)
val_X = tokenizer.texts_to_sequences(val_X)
test_X = tokenizer.texts_to_sequences(test_X)


In [13]:
## Pad the sentences 
train_X = pad_sequences(train_X, maxlen=maxlen)
val_X = pad_sequences(val_X, maxlen=maxlen)
test_X = pad_sequences(test_X, maxlen=maxlen)

In [14]:
## Get the target values
train_y = train_df['target'].values
val_y = val_df['target'].values

In [ ]:
# del model, inp, x
# import gc; gc.collect()
# time.sleep(10)

In [15]:
!ls ../input/embeddings/

ls: cannot access '../input/embeddings/': No such file or directory


In [18]:
EMBEDDING_FILE = '../input/quora-insincere-questions-classification/embeddings/glove.840B.300d/glove.840B.300d.txt'
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE))

all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
embed_size = all_embs.shape[1]

word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector
        
inp = Input(shape=(maxlen,))
x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
x = Bidirectional(CuDNNGRU(64, return_sequences=True))(x)
x = GlobalMaxPool1D()(x)
x = Dense(16, activation="relu")(x)
x = Dropout(0.1)(x)
x = Dense(1, activation="sigmoid")(x)
model = Model(inputs=inp, outputs=x)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  """


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 100)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 100, 300)          15000000  
_________________________________________________________________
bidirectional_1 (Bidirection (None, 100, 128)          140544    
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 16)                2064      
_________________________________________________________________
dropout_1 (Dropout)  

In [19]:
model.fit(train_X, train_y, batch_size=512, epochs=2, validation_data=(val_X, val_y))


Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 162532 samples, validate on 261225 samples
Epoch 1/2
162532/162532 [==============================] - 21s 127us/step - loss: 0.2984 - acc: 0.8787 - val_loss: 0.1850 - val_acc: 0.9233
Epoch 2/2
162532/162532 [==============================] - 18s 108us/step - loss: 0.2216 - acc: 0.9170 - val_loss: 0.1840 - val_acc: 0.9227


In [20]:
pred_glove_val_y = model.predict([val_X], batch_size=1024, verbose=1)
for thresh in np.arange(0.1, 0.501, 0.01):
    thresh = np.round(thresh, 2)
    print("F1 score at threshold {0} is {1}".format(thresh, metrics.f1_score(val_y, (pred_glove_val_y>thresh).astype(int))))

261225/261225 [==============================] - 4s 17us/step
F1 score at threshold 0.1 is 0.404996627406216
F1 score at threshold 0.11 is 0.4146860777686105
F1 score at threshold 0.12 is 0.4231738035264484
F1 score at threshold 0.13 is 0.43121829175064286
F1 score at threshold 0.14 is 0.43911360639700564
F1 score at threshold 0.15 is 0.4463179777389597
F1 score at threshold 0.16 is 0.45283905648738354
F1 score at threshold 0.17 is 0.45897822420560885
F1 score at threshold 0.18 is 0.464845702858701
F1 score at threshold 0.19 is 0.4708216081561102
F1 score at threshold 0.2 is 0.4762929354350843
F1 score at threshold 0.21 is 0.4813165292649799
F1 score at threshold 0.22 is 0.4856202466926567
F1 score at threshold 0.23 is 0.49060937929695714
F1 score at threshold 0.24 is 0.49540833865344003
F1 score at threshold 0.25 is 0.500041372234266
F1 score at threshold 0.26 is 0.5043236824058742
F1 score at threshold 0.27 is 0.5084201814097259
F1 score at threshold 0.28 is 0.512854655331353
F1 scor

In [23]:
pred_glove_train_y = model.predict([train_X], batch_size=1024, verbose=1)

162532/162532 [==============================] - 3s 17us/step


In [ ]:
pred_glove_test_y = model.predict([test_X], batch_size=1024, verbose=1)

In [24]:
del word_index, embeddings_index, all_embs, embedding_matrix, model, inp, x
import gc; gc.collect()
time.sleep(10)

In [25]:
EMBEDDING_FILE = '../input/quora-insincere-questions-classification/embeddings/wiki-news-300d-1M/wiki-news-300d-1M.vec'
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE) if len(o)>100)

all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
embed_size = all_embs.shape[1]

word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector
        
inp = Input(shape=(maxlen,))
x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
x = Bidirectional(CuDNNGRU(64, return_sequences=True))(x)
x = GlobalMaxPool1D()(x)
x = Dense(16, activation="relu")(x)
x = Dropout(0.1)(x)
x = Dense(1, activation="sigmoid")(x)
model = Model(inputs=inp, outputs=x)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  """


In [26]:
model.fit(train_X, train_y, batch_size=512, epochs=2, validation_data=(val_X, val_y))

Train on 162532 samples, validate on 261225 samples
Epoch 1/2
162532/162532 [==============================] - 19s 119us/step - loss: 0.2992 - acc: 0.8768 - val_loss: 0.2403 - val_acc: 0.8975
Epoch 2/2
162532/162532 [==============================] - 17s 107us/step - loss: 0.2124 - acc: 0.9188 - val_loss: 0.2370 - val_acc: 0.9011


In [27]:
pred_fasttext_val_y = model.predict([val_X], batch_size=1024, verbose=1)
for thresh in np.arange(0.1, 0.501, 0.01):
    thresh = np.round(thresh, 2)
    print("F1 score at threshold {0} is {1}".format(thresh, metrics.f1_score(val_y, (pred_fasttext_val_y>thresh).astype(int))))

261225/261225 [==============================] - 4s 17us/step
F1 score at threshold 0.1 is 0.33945514779196556
F1 score at threshold 0.11 is 0.3483631871525633
F1 score at threshold 0.12 is 0.3568477879511527
F1 score at threshold 0.13 is 0.36481005029784674
F1 score at threshold 0.14 is 0.3721470110980824
F1 score at threshold 0.15 is 0.37922857281130523
F1 score at threshold 0.16 is 0.38584075160900544
F1 score at threshold 0.17 is 0.39241348992024117
F1 score at threshold 0.18 is 0.39897743976481115
F1 score at threshold 0.19 is 0.40491722327624297
F1 score at threshold 0.2 is 0.4109277778511698
F1 score at threshold 0.21 is 0.4160823691866311
F1 score at threshold 0.22 is 0.4218048182931809
F1 score at threshold 0.23 is 0.4270137470325181
F1 score at threshold 0.24 is 0.4320709579037201
F1 score at threshold 0.25 is 0.43678356358483883
F1 score at threshold 0.26 is 0.441641917387624
F1 score at threshold 0.27 is 0.4462224287394543
F1 score at threshold 0.28 is 0.45028346502952143
F

In [28]:
pred_fasttext_train_y = model.predict([train_X], batch_size=1024, verbose=1)

162532/162532 [==============================] - 3s 16us/step


In [29]:
del word_index, embeddings_index, all_embs, embedding_matrix, model, inp, x
import gc; gc.collect()
time.sleep(10)

In [30]:
EMBEDDING_FILE = '../input/quora-insincere-questions-classification/embeddings/paragram_300_sl999/paragram_300_sl999.txt'
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE, encoding="utf8", errors='ignore'))

all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
embed_size = all_embs.shape[1]

word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector
        
inp = Input(shape=(maxlen,))
x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
x = Bidirectional(CuDNNGRU(64, return_sequences=True))(x)
x = GlobalMaxPool1D()(x)
x = Dense(16, activation="relu")(x)
x = Dropout(0.1)(x)
x = Dense(1, activation="sigmoid")(x)
model = Model(inputs=inp, outputs=x)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  """


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 100)               0         
_________________________________________________________________
embedding_3 (Embedding)      (None, 100, 300)          15000000  
_________________________________________________________________
bidirectional_3 (Bidirection (None, 100, 128)          140544    
_________________________________________________________________
global_max_pooling1d_3 (Glob (None, 128)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 16)                2064      
_________________________________________________________________
dropout_3 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 17        
Total para

In [31]:
model.fit(train_X, train_y, batch_size=512, epochs=2, validation_data=(val_X, val_y))

Train on 162532 samples, validate on 261225 samples
Epoch 1/2
162532/162532 [==============================] - 21s 127us/step - loss: 0.3029 - acc: 0.8762 - val_loss: 0.2188 - val_acc: 0.9074
Epoch 2/2
162532/162532 [==============================] - 18s 113us/step - loss: 0.2278 - acc: 0.9138 - val_loss: 0.2065 - val_acc: 0.9121


In [32]:
pred_paragram_val_y = model.predict([val_X], batch_size=1024, verbose=1)
for thresh in np.arange(0.1, 0.501, 0.01):
    thresh = np.round(thresh, 2)
    print("F1 score at threshold {0} is {1}".format(thresh, metrics.f1_score(val_y, (pred_paragram_val_y>thresh).astype(int))))


261225/261225 [==============================] - 5s 18us/step
F1 score at threshold 0.1 is 0.36953866391055157
F1 score at threshold 0.11 is 0.3788502638283486
F1 score at threshold 0.12 is 0.38733231028166915
F1 score at threshold 0.13 is 0.3957932083122149
F1 score at threshold 0.14 is 0.4032339434706681
F1 score at threshold 0.15 is 0.4102672385351369
F1 score at threshold 0.16 is 0.4168884587466731
F1 score at threshold 0.17 is 0.42356095425694906
F1 score at threshold 0.18 is 0.4295402746122631
F1 score at threshold 0.19 is 0.43535571644297666
F1 score at threshold 0.2 is 0.44101389087784626
F1 score at threshold 0.21 is 0.44646567381038865
F1 score at threshold 0.22 is 0.451707935011563
F1 score at threshold 0.23 is 0.4569338707269741
F1 score at threshold 0.24 is 0.46155008537709474
F1 score at threshold 0.25 is 0.4659176029962546
F1 score at threshold 0.26 is 0.47077389609184017
F1 score at threshold 0.27 is 0.47546933667083857
F1 score at threshold 0.28 is 0.47940548659973115


In [33]:
pred_paragram_train_y = model.predict([train_X], batch_size=1024, verbose=1)

162532/162532 [==============================] - 3s 17us/step


In [34]:
del word_index, embeddings_index, all_embs, embedding_matrix, model, inp, x
import gc; gc.collect()
time.sleep(10)

In [35]:
pred_val_y = 0.33*pred_glove_val_y + 0.33*pred_fasttext_val_y + 0.34*pred_paragram_val_y 
for thresh in np.arange(0.1, 0.501, 0.01):
    thresh = np.round(thresh, 2)
    print("F1 score at threshold {0} is {1}".format(thresh, metrics.f1_score(val_y, (pred_val_y>thresh).astype(int))))


F1 score at threshold 0.1 is 0.3621718855623919
F1 score at threshold 0.11 is 0.3719756042893039
F1 score at threshold 0.12 is 0.38080825201578733
F1 score at threshold 0.13 is 0.3893223513650967
F1 score at threshold 0.14 is 0.397594793794162
F1 score at threshold 0.15 is 0.4049659529296016
F1 score at threshold 0.16 is 0.4120885649780172
F1 score at threshold 0.17 is 0.4189543626508131
F1 score at threshold 0.18 is 0.4254451951746588
F1 score at threshold 0.19 is 0.4320963614109269
F1 score at threshold 0.2 is 0.438169232942506
F1 score at threshold 0.21 is 0.44423093469469704
F1 score at threshold 0.22 is 0.44971189104243064
F1 score at threshold 0.23 is 0.4551400351379682
F1 score at threshold 0.24 is 0.4603207852055779
F1 score at threshold 0.25 is 0.4656346185920277
F1 score at threshold 0.26 is 0.47045458034603993
F1 score at threshold 0.27 is 0.4753660128687327
F1 score at threshold 0.28 is 0.48030839430414607
F1 score at threshold 0.29 is 0.48529528844163494
F1 score at thresh

In [36]:
pred_train_y = 0.33*pred_glove_train_y + 0.33*pred_fasttext_train_y + 0.34*pred_paragram_train_y 
for thresh in np.arange(0.1, 0.501, 0.01):
    thresh = np.round(thresh, 2)
    print("F1 score at threshold {0} is {1}".format(thresh, metrics.f1_score(val_y, (pred_val_y>thresh).astype(int))))


F1 score at threshold 0.1 is 0.3621718855623919
F1 score at threshold 0.11 is 0.3719756042893039
F1 score at threshold 0.12 is 0.38080825201578733
F1 score at threshold 0.13 is 0.3893223513650967
F1 score at threshold 0.14 is 0.397594793794162
F1 score at threshold 0.15 is 0.4049659529296016
F1 score at threshold 0.16 is 0.4120885649780172
F1 score at threshold 0.17 is 0.4189543626508131
F1 score at threshold 0.18 is 0.4254451951746588
F1 score at threshold 0.19 is 0.4320963614109269
F1 score at threshold 0.2 is 0.438169232942506
F1 score at threshold 0.21 is 0.44423093469469704
F1 score at threshold 0.22 is 0.44971189104243064
F1 score at threshold 0.23 is 0.4551400351379682
F1 score at threshold 0.24 is 0.4603207852055779
F1 score at threshold 0.25 is 0.4656346185920277
F1 score at threshold 0.26 is 0.47045458034603993
F1 score at threshold 0.27 is 0.4753660128687327
F1 score at threshold 0.28 is 0.48030839430414607
F1 score at threshold 0.29 is 0.48529528844163494
F1 score at thresh

numpy.ndarray

In [62]:
#pred_train_df = pd.DataFrame({"prediction":pred_train_y}, index=[0])
pd.DataFrame(pred_train_y).to_csv("pred_train.csv", header=None, index=None)

In [63]:
#pred_train_df = pd.DataFrame({"prediction":pred_train_y}, index=[0])
pd.DataFrame(pred_val_y).to_csv("pred_val.csv", header=None, index=None)